In [212]:
from langchain.schema import HumanMessage

In [213]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
# from langchain_community.chat_models import ChatOpenAI

from langchain.llms import openai
from typing import TypedDict
from dotenv import load_dotenv
import os

In [214]:
load_dotenv()

True

In [215]:
model = ChatOpenAI(
    model="google/gemma-3n-e2b-it:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7
)

In [217]:
#create a state
class LLMState(TypedDict):
    question:str
    answer:str

In [218]:
def llm_qa(state: LLMState) -> LLMState:
    # Extract the question from the state
    question = state['question']
    
    # Check if model is properly initialized
    if model is None or not hasattr(model, 'invoke'):
        state['answer'] = "Error: Model not properly initialized. Please check your API key and try again."
        return state
    
    # Form a prompt
    prompt = f"Answer the following question: {question}"
    
    # Ask the question to the LLM
    try:
        response = model.invoke([HumanMessage(content=prompt)])
        answer = response.content
    except Exception as e:
        answer = f"Error calling model: {str(e)}"
    
    # Update the answer in the state
    state['answer'] = answer
    
    return state

In [219]:
#create a graph

graph=StateGraph(LLMState)

#add nodes
graph.add_node('llm_qa',llm_qa)

#add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

#compile
workflow=graph.compile()


In [222]:
#execute
initial_state={'question':'How far is moon from earth'}

workflow.invoke(initial_state)

{'question': 'How far is moon from earth',
 'answer': "The average distance from the Moon to the Earth is about 384,400 kilometers, or 238,855 miles. This distance varies slightly throughout the Moon's orbit, with the closest approach (perigee) being around 363,104 kilometers and the farthest point (apogee) being about 405,696 kilometers."}